# This is my follow-along file for chapter 2 of "Hands-On Machine Learning with Scikit-Learn & Tensorflow" by Aurelien Geron.

In [86]:
#DOWNLOADING THE HOUSING DATASET
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()


fetch_housing_data()

In [87]:
#LOADING THE HOUSING DATASET USING PANDAS

import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

load_housing_data(HOUSING_PATH)

In [71]:
housing = load_housing_data()
housing.head()

In [88]:
# Info about the columns
housing.info()

In [89]:
# counting the different values present in ocean_proximity
housing["ocean_proximity"].value_counts()

In [90]:
#describe
housing.describe()

In [91]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [92]:
#Function to separate test set
import numpy as np
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]


train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set), "train +", len(test_set), "test")

In [93]:
import hashlib

def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[-in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index() # adds an 'index' column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index") # the id is not that much stable, addition of new rows may alter the ids

In [94]:
housing_with_id["id"] = housing["longitude"] + 1000 + housing["latitude"] # --------> More stable id, as it relies on Longitude and Latitude which are not going to change.
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [11]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [12]:
housing["income_cat"] = np.ceil(housing["median_income"]/1.5)
housing["income_cat"] = housing["income_cat"].where(
    housing["income_cat"] < 5, 5.0
)


In [95]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

housing["income_cat"].value_counts() / len(housing)

In [96]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [72]:
housing.head()

In [16]:
housing = strat_train_set.copy()

In [97]:
housing.plot(kind="scatter", x="longitude", y="latitude")

In [98]:
Visualization with alpha
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [99]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
            )
plt.legend()

In [73]:
housing.head(2)

In [21]:
# housing = housing.drop(['ocean_proximity'], axis=1)

In [22]:
housing.head(2)

In [23]:
corr_matrix = housing.corr()

In [24]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [74]:
housing.head()

In [100]:
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [101]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [75]:
housing.head()

In [76]:
housing.drop(['ocean_proximity'], axis=1).head()

In [30]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [31]:
corr_matrix = housing.drop(['ocean_proximity'], axis=1).corr()

In [102]:
corr_matrix["median_house_value"]

In [33]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [34]:
housing.dropna(subset=["total_bedrooms"]) #option 1

housing.drop("total_bedrooms", axis=1) #option 2

median = housing["total_bedrooms"].median() #option 3
housing["total_bedrooms"].fillna(median, inplace=True)

In [35]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [103]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [104]:
imputer.fit(housing_num)

In [105]:
imputer.statistics_

In [106]:
housing_num.median().values

In [107]:
X = imputer.transform(housing_num)
print(X[0])

In [108]:
housing_tr=pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head(4)

In [109]:
# housing.head()

In [110]:
#Using Label-Encoder to encode ocean_proximity
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded #----------> This method alone is not reliable in the case of housing data. Because, it is pointless to represent it in order because there isn't any natural ranking. We are going to use One-Hot Encoder over this for the purpose
print(encoder.classes_)

In [111]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1, 1))
housing_cat_1hot

In [112]:
housing_cat_1hot.toarray()

In [113]:
# Whatever we did above, wasn't short at all. We can use a much more "Lilliputer" approach.
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

In [114]:
#Custom Transformers
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self #nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, rooms_ix]

        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
        

In [48]:
# Feature Scaling
# (This step is generally not required but still beneficial to do in some cases.)

#Transformation Pipelines
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([                                          #_________> And this hot stuff is called a transformation pipeline
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)


In [49]:
# This pipeline which we are using can only be fed NumPy arrays. We need a method to cook our Pandas DataFrame in order to break it down to NumPy array

from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# Using DataFrameSelector in order to convert different attributes' columns for pipelines.

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([                                         #This is for Num-Num Style data
    ('selector', DataFrameSelector(num_attribs)),                 #This is a simple DataFrame to NumPy array converter
    ('imputer', SimpleImputer(strategy="median")),                #This is Simple Imputer imputing missing values using its median strategy
    ('attribs_adder', CombinedAttributesAdder()),                 #This is where new attributes like rooms_per_household get manufactured and added
    ('std_scaler', StandardScaler()),                              #This is a Scaler which will scale values down for simpler calculations
     ])

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([                                         #This is for Cat-Cat data
    ('selector', DataFrameSelector(cat_attribs)),                 #This is a simple DataFrame to NumPy array converter
    ('OneHotEncoder', OneHotEncoder()),                        #This handles the categorical data by binarization
])

In [50]:
#Combining the two pipelines
# Now it is time to combine the two pipelines and create a bigger pipeline of data. We will be using FeatureUnion class for the purpose.

from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared
housing_prepared.shape

(16512, 16)

In [77]:
Select and Train a Model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()                   #-->This model underfits the data
lin_reg.fit(housing_prepared, housing_labels)

In [78]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [79]:
# Checking the RMSE(root mean square error)
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [80]:
# Not a very satisfactory score, let's try something different. Maybe, a different model. 
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()               #--->This model overfits the data
tree_reg.fit(housing_prepared, housing_labels)

In [81]:
housing_predictions = tree_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [82]:
#Better Evaluation Using Cross-Validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [83]:
# Let's look at the results
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())

display_scores(scores)                 # The model is performing really bad.

In [84]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_scores)

In [59]:
# Random Forest 
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

np.float64(18362.524402741095)

In [85]:
Fine-Tuning the model
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators' : [3, 10, 30], 'max_features' : [2, 4, 6, 8]},
    {'bootstrap' : [False], 'n_estimators' : [3, 10], 'max_features' :[2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

grid_search.best_params_

In [65]:
# These two lines are gonna save you a lot of time. The two lines return the full fledged fine-tuned model, automatically calculating the best_params.
grid_search.best_estimator_
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None, max_features=8, max_leaf_nodes=None, min_impurity_decrease=1e-07, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1, oob_score=False, random_state=42, verbose=0, warm_start=False)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",30
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'mse'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",8
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curr

In [66]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

65066.764060685935 {'max_features': 2, 'n_estimators': 3}
55770.630804477376 {'max_features': 2, 'n_estimators': 10}
53057.88590201809 {'max_features': 2, 'n_estimators': 30}
61515.567627645934 {'max_features': 4, 'n_estimators': 3}
52986.197251586935 {'max_features': 4, 'n_estimators': 10}
50383.9444181586 {'max_features': 4, 'n_estimators': 30}
58111.868400410705 {'max_features': 6, 'n_estimators': 3}
51999.401815276826 {'max_features': 6, 'n_estimators': 10}
49733.374379362045 {'max_features': 6, 'n_estimators': 30}
58309.60881517763 {'max_features': 8, 'n_estimators': 3}
51743.02342694998 {'max_features': 8, 'n_estimators': 10}
49577.57238824816 {'max_features': 8, 'n_estimators': 30}
62189.69224131646 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54112.02200736774 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
58681.21702935818 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
51823.81125236615 {'bootstrap': False, 'max_features': 3, 'n_estimator

In [67]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([7.65322493e-02, 7.44172429e-02, 3.97815996e-02, 1.47622240e-02,
       1.52353536e-02, 1.59570348e-02, 1.46801324e-02, 3.45330101e-01,
       4.25041532e-02, 1.47342830e-01, 4.05561610e-02, 1.14900358e-02,
       1.56126704e-01, 3.57385448e-05, 2.11123087e-03, 3.13720899e-03])

In [68]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(np.float64(0.3453301007081541), 'median_income'),
 (np.float64(0.15612670406321366), np.str_('INLAND')),
 (np.float64(0.14734283023835154), 'pop_per_hhold'),
 (np.float64(0.07653224929301934), 'longitude'),
 (np.float64(0.07441724285829229), 'latitude'),
 (np.float64(0.04250415324173311), 'rooms_per_hhold'),
 (np.float64(0.0405561610179327), 'bedrooms_per_room'),
 (np.float64(0.03978159957311375), 'housing_median_age'),
 (np.float64(0.01595703479970965), 'population'),
 (np.float64(0.015235353583852473), 'total_bedrooms'),
 (np.float64(0.014762224036610462), 'total_rooms'),
 (np.float64(0.014680132386395867), 'households'),
 (np.float64(0.011490035793746823), np.str_('<1H OCEAN')),
 (np.float64(0.0031372089940830423), np.str_('NEAR OCEAN')),
 (np.float64(0.002111230866980616), np.str_('NEAR BAY')),
 (np.float64(3.573854481062883e-05), np.str_('ISLAND'))]

In [70]:
# Now, it is time to evaluate the system on the test set
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

np.float64(47358.729943150494)